In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import sys
from pathlib import Path
import tikzplotlib

try:
    sys.path.append(str(Path().cwd().parent))
except IndexError:
    pass

from cqp.cqp_regression import *

In [ ]:
prefix_cycle = cycle(MODE_PREFIXES)

In [ ]:
EVAL_DIR = "sa_ta"
MODE_PREFIX = next(prefix_cycle)
BASE_KEY = MODE_PREFIX + "base"
FILTER = MODE_PREFIX + "median"

print(f"Current cycle: {MODE_PREFIX}")

In [ ]:
data_path = Path("average.csv")
data_path = data_path if data_path.exists() else Path.cwd().parent.parent / "config" / EVAL_DIR / "SATA" / "average.csv"
df = read_df(data_path)

In [ ]:
cycol, filter_keys = plot_bitrate(BASE_KEY, df, FILTER)

tikzplotlib.save("rate-over-qp-filtered.tex")

In [ ]:
base, lines, _ = get_qp_match(BASE_KEY, cycol, df, FILTER, filter_keys)

tikzplotlib.save("linear-regression-2d.tex")

In [ ]:
X, f, linear_regression, y = fit_plane(base, lines, MODE_PREFIX)

plt.savefig("linear-regression-3d.pdf")

$FS = f(QP_{i}, QP_{SF}) = a + b\cdot QP_i + c\cdot QP_{SF}$

In [ ]:
print(f"FS = {linear_regression.intercept_} + {linear_regression.coef_[0]} * QP_i + {linear_regression.coef_[1]} * QP_SF")

In [ ]:
linear_regression_mae = mean_absolute_error(y, linear_regression.predict(X))
linear_regression_pc = get_pc(y, linear_regression.predict(X))
plane_error = mean_absolute_error(y, predict_with_plane_equation(
    linear_regression.intercept_, linear_regression.coef_[0], linear_regression.coef_[1], X))

print(F"MAE linear regression: {linear_regression_mae:.5f} with PC {linear_regression_pc:.5f}")
print(f"Plane equation MAE: {plane_error:.5f}")

In [ ]:
model_params_list, eval_data = zip(*[evaluate_block(df, prefix) for prefix in tqdm(MODE_PREFIXES)])

In [ ]:
average_model = get_average_model_params(model_params_list)
st_model = get_st_model(model_params_list)

In [ ]:
evaluation_data = pd.concat(eval_data)

avg_prediction = average_model.plane.predict(
    evaluation_data[["qp_i", "qp_sf"]].to_numpy()
)
bitrate_evaluation(evaluation_data, avg_prediction, df, "average")